In [12]:
import os
import time
import httplib2
import pandas as pd
from googleapiclient.discovery import build
from googleapiclient.errors import HttpError
from dotenv import load_dotenv  # 导入 dotenv 库

# 加载 .env 文件中的环境变量
load_dotenv()

# 从环境变量中读取 API 密钥
API_KEY = os.getenv('API_KEY')

# 设置代理信息（使用本地代理地址和端口）
proxy_info = httplib2.ProxyInfo(
    proxy_type=httplib2.socks.PROXY_TYPE_HTTP, 
    proxy_host='127.0.0.1', 
    proxy_port=1087
)

# 配置带代理的 HTTP 客户端
http = httplib2.Http(proxy_info=proxy_info, timeout=60)
youtube = build('youtube', 'v3', developerKey=API_KEY, http=http)

# 定义获取播放列表中的视频 ID 的函数
def get_playlist_video_ids(playlist_id):
    video_ids = []
    next_page_token = None

    while True:
        try:
            response = youtube.playlistItems().list(
                part='contentDetails',
                playlistId=playlist_id,
                maxResults=50,  # 每页最多50个视频
                pageToken=next_page_token
            ).execute()

            for item in response['items']:
                video_ids.append(item['contentDetails']['videoId'])

            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break  # 没有更多视频时退出循环

            time.sleep(1)  # 防止请求过于频繁

        except HttpError as e:
            print("Error fetching playlist videos:", e)
            break

    return video_ids

# 获取单个视频的所有评论
def get_video_comments(video_id, max_retries=3):
    comments = []
    next_page_token = None
    retries = 0

    while True:
        try:
            response = youtube.commentThreads().list(
                part='snippet',
                videoId=video_id,
                pageToken=next_page_token,
                maxResults=100,  # 每次请求最多100条评论
                textFormat="plainText"
            ).execute()

            for item in response['items']:
                comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
                comments.append(comment)

            next_page_token = response.get('nextPageToken')
            if not next_page_token:
                break  # 没有更多评论时退出循环

            time.sleep(1)  # 防止请求过于频繁

        except HttpError as e:
            retries += 1
            if retries >= max_retries:
                print(f"Failed to fetch comments for video {video_id} after {max_retries} retries.")
                break
            print(f"Retrying {retries} of {max_retries} in 10 seconds...")
            time.sleep(10)

    return comments

# 播放列表 ID 列表
playlists = {
    "shanhaiqing": "PLUg6CGAXr7pktTjLdLYzBtgpjUMgUfkaZ",  # 上海情播放列表
    "fengchuoibanxia": "PLAYWca_88zhG3Fco5TTD61kUneKLpmWzH"  # 风吹半夏播放列表
}

# 爬取评论并保存到 Excel
for collection_name, playlist_id in playlists.items():
    video_ids = get_playlist_video_ids(playlist_id)
    all_comments = []

    print(f"Total videos in playlist '{collection_name}': {len(video_ids)}")

    for video_id in video_ids:
        print(f"Fetching comments for video ID: {video_id}")
        comments = get_video_comments(video_id)
        all_comments.extend(comments)

    # 创建 DataFrame 并保存为 Excel 文件
    df = pd.DataFrame(all_comments, columns=["Comment"])
    file_name = f"{collection_name}_comments.xlsx"
    df.to_excel(file_name, index=False)

    # 打印评论总数
    print(f"{file_name} 评论总数: {len(all_comments)}")


Total videos in playlist 'shanhaiqing': 23
Fetching comments for video ID: SpZqeVqtMKk
Fetching comments for video ID: QJb3PNe_XCA
Fetching comments for video ID: 0c0DKBkjaSo
Fetching comments for video ID: ZB4B5arN75s
Fetching comments for video ID: aTUz4kupqPE
Fetching comments for video ID: dXzpn5F2ADU
Fetching comments for video ID: GZCOZKcVdc8
Fetching comments for video ID: ZGHj0_txuyE
Fetching comments for video ID: -6jCvwn85Fo
Fetching comments for video ID: 6K40j75NaSw
Fetching comments for video ID: C5NV-Ls-e6U
Fetching comments for video ID: 3bTpZ1DTX48
Fetching comments for video ID: o6NpsJwuOtk
Fetching comments for video ID: 8iLynHPIRio
Fetching comments for video ID: qrlQFSEn9rs
Fetching comments for video ID: CFOjxpHHi-8
Fetching comments for video ID: AKCjC4yluEE
Fetching comments for video ID: WoxeMTag9JM
Fetching comments for video ID: JAk4LgdMP7E
Fetching comments for video ID: Ek_CVJHsC2s
Fetching comments for video ID: WNFt0tUnIvw
Fetching comments for video ID: V

## 分别将不为中文和中文的评论保存到不同的 Excel 文件中：

In [13]:
import pandas as pd
import os
import re

# 定义判断中文的函数
def is_chinese(string):
    pattern = re.compile(r'[\u4e00-\u9fa5]')
    return bool(pattern.search(string))

# 文件路径
collections = ["fengchuoibanxia", "shanhaiqing"]
for collection in collections:
    file_path = f"{collection}_comments.xlsx"
    
    # 读取评论文件
    df = pd.read_excel(file_path)

    # 筛选评论
    df_chinese = df[df['Comment'].apply(is_chinese)]  # 中文评论
    df_ncn = df[~df['Comment'].apply(is_chinese)]     # 非中文评论

    # 保存结果到新的 Excel 文件
    df_chinese.to_excel(f"{collection}_comments_CN.xlsx", index=False)
    df_ncn.to_excel(f"{collection}_comments_NCN.xlsx", index=False)

    # 打印结果
    print(f"{collection}_comments_CN.xlsx 总数: {len(df_chinese)}")
    print(f"{collection}_comments_NCN.xlsx 总数: {len(df_ncn)}")


fengchuoibanxia_comments_CN.xlsx 总数: 850
fengchuoibanxia_comments_NCN.xlsx 总数: 86
shanhaiqing_comments_CN.xlsx 总数: 4843
shanhaiqing_comments_NCN.xlsx 总数: 52


In [14]:
import pandas as pd

# 定义文件路径
collections = ["fengchuoibanxia", "shanhaiqing"]
for collection in collections:
    # 读取非中文评论的 Excel 文件
    df_ncn = pd.read_excel(f"{collection}_comments_NCN.xlsx")

    # 保存为 TXT 文件
    txt_file_path = f"{collection}_comments_NCN.txt"
    with open(txt_file_path, "w", encoding="utf-8") as f:
        for comment in df_ncn["Comment"]:
            f.write(comment + "\n")  # 每个评论单独一行

    print(f"{txt_file_path} 已保存，包含评论总数: {len(df_ncn)}")


fengchuoibanxia_comments_NCN.txt 已保存，包含评论总数: 86
shanhaiqing_comments_NCN.txt 已保存，包含评论总数: 52


In [15]:
import pandas as pd

# 定义要写入的文本
text_content = [
    "爱赵丽颖 ❤",
    "主演太漂亮了。",
    "❤❤❤❤❤❤❤❤❤❤❤❤❤",
    "显然，这发生在河南。",
    "赵丽颖，我最喜欢的女演员 ❤❤❤",
    "我 ❤ 赵丽颖",
    "❤❤❤",
    "我喜欢赵丽颖的所有剧 ❤",
    "很好",
    "这部电影的标题是什么🙏？？",
    "👍🏻👍🏻👍🏻💕",
    "嘿",
    "你打算什么时候下载所有剧集？其他网站已经下载了更多剧集，有些网站甚至有全部剧集！",
    "请给我英文翻译😭",
    "请也提供英文字幕。",
    "我相信这将是一部杰作 😊😊😊",
    "能告诉我这部剧的英文标题吗，谢谢",
    "赵丽颖，收视女王的顶级剧 🎉",
    "在某种程度上很高兴王一博被替换，我觉得欧豪更适合这个角色，不想冒犯一博的粉丝，只是说说！",
    "赵丽颖 👍👍👍👍",
    "赵丽颖是优秀的女演员，剧集非常好，非常感谢赵丽颖 👏👏👏👏🙏🙏✨🌟💯💯👍👍👍💪👸",
    "赵丽颖太棒了",
    "👍👍👍👍👍",
    "13:24",
    "我羡慕这三个人的友谊",
    "很少能遇到像许爸这样的父亲。祖母。。。叫自己的孩子这样",
    "她生气的时候太好笑了 😜😜😍😍",
    "非常感谢爱奇艺，希望能上传所有剧集 🙏🙏🙏",
    "赵丽颖 👍👍👍",
    "赵丽颖的剧真棒，非常好的剧集，祝贺 💯💯💯🙏🙏🙏👏👏👏👏👏👸🤞✌✌🥲😅",
    "赵丽颖！永远是最棒的！！ 💯💯💯💯💯🔥🔥🔥🔥🔥⭐️⭐️⭐️⭐️⭐️ ❤️❤️❤️❤️❤️",
    "我去打天门 嘿嘿",
    "🇻🇪🇻🇪🇻🇪🇻🇪🇻🇪🇻🇪🇻🇪🇻🇪🇻🇪🇻🇪",
    "爱奇艺的剧真是太棒了，值得观看，谢谢频道 🇳🇱🇳🇱👑👑🥇🥇🌟😍😍💖🌹💎💎💟💟☯💝💝🌙⭐✅🌷🌷🌷👍👍💐💐💐",
    "赵丽颖的剧很壮观，恭喜 👍👍👍👍💯💯💯👏👏👏👏✌✌✌✌🙏🙏🙏🥰🥰💚💚👸",
    "看了几集，许爸为了报仇有点厚脸皮，看着搞笑但又聪明",
    "我希望我能读懂中文，无法理解所有评论 🙊🙊🙊🙊",
    "她太狠了，离婚和我在一起 😜😜😜😜😂😂😂😂",
    "🔥🔥🔥🔥👍❤️❤️❤️❤️❤️ 很有趣",
    "徐班霞是最棒的 💞💞💞💞💞🌻🌻🌻🌻🌻🌹🌹🌹🌹🌹👍👍👍👍👍🔥🔥🔥🔥🔥🥇🥇🥇🥇🥇💯💯💯💯💯⭐️⭐️⭐️⭐️⭐️🎉🎉🎉🎉🎉🥰👸",
    "赵丽颖的剧真精彩，总是能实现梦想，只要努力工作，带着心去做 👏👏👏👏👌👌💪🌟🌟🌟💯💯✌🙏🙏🙏💚💚💚🤞✌👸😂😂",
    "赵丽颖 👍👍👍👍",
    "陈不是个诚实的朋友",
    "赵丽颖永远是最棒的。",
    "这部剧真是吸引人。赵丽颖和她的两个搭档表现得很出色。",
    "谢谢！这剧真好 👍",
    "那一集大家笑得太夸张了",
    "哇，真不错 🇳🇵",
    "谢谢！",
    "哈哈，她在监狱里放松，真不错 XB 😂😂",
    "这部剧太上瘾了。我等不及要看后面的剧集了。赵丽颖在这里表现得非常棒。所有演员都很出色。非常真实，符合那个时代。我记得当时的所有时尚。很大的耳环、厚底鞋和波浪卷发…… ❤❤❤",
    "电影太好了 👍",
    "真是精彩，每次都越来越有趣，我们继续 🤟👏👏👏👏💋🌹",
    "精彩的剧！演员之间的友情令人感动。尽管他们面临困难，但他们互相支持。💥💥💥💥💥",
    "徐班霞为了现在的自己努力奋斗。她有权要求应有的尊重。赞美赵丽颖的演技",
    "这一集太感人了。徐对陈的女友非常直接，因为徐把陈的幸福放在第一位。",
    "我喜欢徐班霞对待感情的思考。她考虑得很周到，也有要求",
    "L",
    "😭😭😭😭😭😭😭😭",
    "😍😍😍😍😍",
    "🎉",
    "剧名",
    "祝这个频道的所有人圣诞快乐！",
    "🎅👼🧑‍🎄🤶",
    "祝这部剧的作者圣诞快乐 ❤🎅❤🤶❤🧑‍🎄❤",
    "祝所有艺术家圣诞快乐！！！",
    "🌺🧑‍🎄🌺🎅🌺🤶🌺",
    "祝大家圣诞快乐！！！！",
    "❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤❤",
    "角色“卖哈”也有很多和现实中的颖宝相似的地方",
    "颖宝在生活中也很像“卖哈”，聪明、勇敢，敢说敢做，坚韧不拔，但很有人情味",
    "虽然现在她遇到了一个像赵瑟瑟那样的好人……但她仍然不相信爱情……所以常常犹豫",
    "这部剧中每个角色，从主角到配角都表现得很出色，像许爸和王全，虽然出现得不多，但表现得太棒了",
    "多么精彩的故事！！！我爱上了。",
    "🇧🇷🇧🇷🇧🇷❤🇧🇷🇧🇷🇧🇷",
    "蓝鸟",
    "我羡慕颖宝能和那么多帅哥搭档",
    "喜欢",
    "霍建华",
    "刘恺威",
    "陆毅",
    "陈伟霆",
    "张翰",
    "林更新",
    "李光洁",
    "徐是个很正直的人，赵丽颖总是以她出色的表演打动我",
    "郭麒麟的演技很到位，情感真挚",
    "👍🏻👍🏻👍🏻🈵️☺️",
    "陆熙和许班霞这一对太幸福了",
    "“卖哈”和赵瑟瑟这一对的默契。",
    "他们的搭档太好了。",
    "我在等……",
    "👍🏻👍🏻👍🏻💕",
    "哎呀，这一集太紧张了……",
    "最后，这个父亲终于成为了好父亲……",
    "太美丽也太感人了，我看过很多赵丽颖的剧，她是一位了不起且出色的女演员，我爱她可爱的笑容。",
    "这是一部意义深远的好电影",
    "👍🏻👍🏻👍🏻💕",
    "我看这部剧真的太惊艳了，赵丽颖太美太棒了，她应该是第一名",
    "这部剧美丽而富有意义，感谢你带来这么棒的剧，赵丽颖作为许班霞永远在我心中💚💛💚💛💚💛👸😘😘😘",
    "好剧……值得学习的教训。祝贺大家……"
]

# 创建DataFrame
df = pd.DataFrame(text_content, columns=["评论"])

# 保存为Excel文件
df.to_excel('fengchuoibanxia_comments_NCN_translated.xlsx', index=False)



In [16]:
import pandas as pd

# 定义要写入的文本
text_content = [
    "这里是对导演孔笙的赞美。我刚开始接触中国电视剧，但最近我喜欢上了他导演的一些剧： 《琅琊榜》《如懿传》《伪装者》《欢乐颂》。",
    "第一集看起来不错。让我从头看到尾 🤿",
    "这是我非常喜欢的第一部方言剧。故事感人，夹杂着一点幽默。我刚开始不太明白，但随着情节的发展，这是一场娱乐的过山车。感谢参与这部有意义的剧的老前辈们。👏",
    "感谢字幕组。",
    "感谢你们做的英文字幕，让我能给孩子看这个。",
    "满怀感激和同情。",
    "👍👍👍😃😃😃",
    "看起来很有趣，但我在等英文字幕。",
    "我希望他们会播出带有英文字幕的《矿镇》。请加上英文字幕。",
    "看到水华满怀决心地背着丈夫和女儿，流下了泪水。我知道她经历了多少困难。而且她多年后再次见到马德富？！ 🥹🥹🥹",
    "水华太坚强了。为了新村走了七天七夜，背着残疾的丈夫和孩子。真疯狂。",
    "当她说她不知道自己有多重要时...太伤心了 😭",
    "水华让我哭了三次。多么了不起的女人，她的微笑！比阳光还要明亮。演员们都表现得很棒！",
    "感谢字幕组！！！我终于可以观看了！！哇哦 :) 我爱这部剧，感觉如此真实。",
    "如果可以的话，请把前两集的英文字幕加上。",
    "为什么在第二集之后停止更新英文字幕？我的外国朋友也想看！求求你了！！谢谢",
    "剩下的集数会有英文字幕吗？",
    "好剧",
    "帮帮我，我在网上找不到第六集之后的字幕剧集。我真的想看这部剧到最后😭😭前几集太好了。请大家帮我找找有字幕的剧集！！",
    "我们真的不能没有水。尊重水 😭",
    "非常感谢！我看到现在所有剧集都有英文字幕。我很感激！",
    "我爱中国@！@",
    "不能责怪那些对蘑菇项目持怀疑态度的村民。他们经历了如此艰难的生活。所以，有时候还是看看领导们的项目是否有效更好。",
    "我非常喜欢这些角色。 🥹😭❤️‍🔥",
    "优秀的剧！",
    "我在等",
    "真实的节目，真正反映了普通人的生活",
    "2:41 当陈先生意识到他也想和村民们一起合影时。 😭",
    "我一看到他的表情，就知道那是他想要的。 😭❤️",
    "而且，大友叔叔真是卑鄙。我理解村民们不想失去所有蘑菇和无法高价出售的心情，但他的自私让人愤怒。他从不为自己的行为负责，指责其他人，然后试图煽动大家。他让我想起了我的父亲。🫥",
    "我是巴西人，最近开始看一些中国制作的剧，这部让我非常感动。祝贺你们。",
    "谢谢你把这部剧放到YouTube上。这是我最近看过的最好的东西之一！",
    "25:38 每当我看到中国剧中桌子上摆着看起来美味的食物时，我就开始饿了。",
    "凌教授是个英雄。",
    "17:35 xswl",
    "老师白连自己都买不起新眼镜，女儿却随口说他自私。真悲伤。",
    "麦苗看不到大局，她父亲的行为是为了更大的利益。白被告知让他的学校参加歌唱比赛。虽然村子贫困，但没有资金为活动制作制服。还有什么办法参与？",
    "她只考虑自己，如何受到电脑销售的影响。她没有做得很好或足够——连提前告诉父亲她回家的事都做不到，或者带点礼物去看他。捐赠给学校的电脑却没有培训如何使用，简直无用。",
    "😭😭😭😭😭😭😭",
    "1:45 宋云辉为了建设他的国家双重工作。我想知道他是否曾经休假一周，因为东哈没有他就会崩溃。现在他必须去救助矿镇 😂",
    "王凯，你今天能够达到这样的水平，真是辛苦你了。致敬。",
    "🎉🎉🎉🎉👍👍",
    "我爱这个！！！！我每集都流泪！",
    "结局",
    "😭😭",
    "不错的结局"
]

# 创建DataFrame
df = pd.DataFrame(text_content, columns=["评论"])

# 保存为Excel文件
df.to_excel('shanhaiqing_comments_NCN_translated.xlsx', index=False)

print("文本已成功保存为 shanhaiqing_comments_NCN_translated.xlsx")


文本已成功保存为 shanhaiqing_comments_NCN_translated.xlsx


In [17]:
import pandas as pd

# 合并 shanhaiqing_comments_NCN 和 shanhaiqing_comments_NCN_translated.xlsx
shanhaiqing_comments = pd.read_excel('shanhaiqing_comments_CN.xlsx', header=None)
shanhaiqing_translated = pd.read_excel('shanhaiqing_comments_NCN_translated.xlsx', header=None)

# 合并数据，确保都在第一列
shanhaiqing_combined = pd.concat([shanhaiqing_comments, shanhaiqing_translated], axis=0, ignore_index=True)

# 保存合并后的文件
shanhaiqing_combined.to_excel('shanhaiqing_comments_all_translated.xlsx', index=False, header=False)

# 合并 fengchuoibanxia_comments_NCN 和 fengchuoibanxia_comments_NCN_translated.xlsx
fengchuoibanxia_comments = pd.read_excel('fengchuoibanxia_comments_CN.xlsx', header=None)
fengchuoibanxia_translated = pd.read_excel('fengchuoibanxia_comments_NCN_translated.xlsx', header=None)

# 合并数据，确保都在第一列
fengchuoibanxia_combined = pd.concat([fengchuoibanxia_comments, fengchuoibanxia_translated], axis=0, ignore_index=True)

# 保存合并后的文件
fengchuoibanxia_combined.to_excel('fengchuoibanxia_comments_all_translated.xlsx', index=False, header=False)

print("合并完成，文件已成功保存！")


合并完成，文件已成功保存！
